In [40]:
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy import signal
from tqdm import tqdm
from shapely.wkt import loads

from shapely.geometry import Point

## On fusionne les données débit météo

In [41]:
stations_debit = pd.read_csv("../Data/Extraction_Hydro/Stations.csv")
stations_debit.head()

,Code station,Libelle station,Regime influencé,Altitude du zéro de l'échelle,geometry,Ordre,Cours eau
0,O0200020,La Garonne [partielle] à Saint-Gaudens [Valent...,pas ou faiblement,357.00,POINT (0.7068033357391884 43.09775183863328),0,Garonne
1,O2620010,La Garonne à Verdun-sur-Garonne,pas ou faiblement,90.00,POINT (1.242176195808334 43.85463266047086),1,Garonne
2,O6140010,La Garonne à Lamagistère,pas ou faiblement,46.00,POINT (0.8312973485763152 44.12107242091946),2,Garonne
3,O9000010,La Garonne à Tonneins,pas ou faiblement,0.00,POINT (0.2221525107266094 44.41192952559171),3,Garonne
4,K0030020,La Loire à Issarlès [Pont de La Borie],fortement,881.34,POINT (4.048486196061388 44.81973247281849),0,Loire


In [42]:
stations_debit.to_csv("../Data/Base/Stations_Debit.csv",
                     index = False)

In [43]:
stations_meteo = pd.read_csv("../Data/Extraction_Meteo/Stations.csv")
stations_meteo.head()

,ID,Nom,Latitude,Longitude,Altitude,geometry
0,7005,ABBEVILLE,50.136000,1.834000,69,POINT (1.834 50.136)
1,7015,LILLE-LESQUIN,50.570000,3.097500,47,POINT (3.0975 50.57)
2,7027,CAEN-CARPIQUET,49.180000,-0.456167,67,POINT (-0.456167 49.18)
3,7037,ROUEN-BOOS,49.383000,1.181667,151,POINT (1.181667 49.383)
4,7072,REIMS-PRUNAY,49.209667,4.155333,95,POINT (4.155333 49.209667)


In [44]:
stations_meteo.to_csv("../Data/Base/Stations_Meteo.csv",
                     index=False)

In [45]:
mesures_debit = pd.read_csv("../Data/Extraction_Hydro/Mesures.csv")
mesures_debit.head()

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,H0100020,H0400010,H0400020,H0800011,H0800012,H0810010,H1700010,H3930020,H4340020,H8100021
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,25.0,59.9,9.39,25.3,13.7,26.7,55.8,308.0,306.0,703.0
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,25.1,60.1,9.50,25.0,13.5,27.5,56.6,308.0,303.0,710.0
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,25.2,60.1,9.60,24.7,13.4,28.3,57.4,310.0,302.0,716.0
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,25.3,60.1,9.63,24.4,13.3,29.2,58.1,312.0,301.0,724.0
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,25.3,60.1,9.64,24.2,13.2,29.9,58.7,313.0,300.0,742.0


In [46]:
mesures_debit.shape

(48216, 27)

In [47]:
mesures_meteo = pd.read_csv("../Data/Extraction_Meteo/Mesures.csv")
mesures_meteo.head()

,Date,Pression_07005,Direction_vent_07005,Vitesse_vent_07005,Temperature_07005,Humidite_07005,Precipitations_07005,Pression_07015,Direction_vent_07015,Vitesse_vent_07015,...,Vitesse_vent_07747,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Direction_vent_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 00:00:00,98806.666667,133.333333,6.200000,0.4,82.666667,0.0,99116.666667,36.666667,5.500000,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
1,2010-01-01 02:00:00,98806.666667,133.333333,6.200000,0.4,82.666667,0.0,99116.666667,36.666667,5.500000,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
2,2010-01-01 04:00:00,98800.000000,360.000000,6.200000,0.2,82.000000,0.0,99110.000000,30.000000,5.100000,...,11.800000,10.000000,61.000000,0.000000,99180.000000,220.000000,9.800000,14.300000,90.000000,0.000000
3,2010-01-01 06:00:00,98873.333333,126.666667,5.466667,0.2,80.666667,0.0,99156.666667,30.000000,6.166667,...,9.066667,10.533333,50.333333,0.000000,99193.333333,240.000000,6.666667,12.900000,78.000000,0.533333
4,2010-01-01 08:00:00,98980.000000,126.666667,5.300000,0.2,80.333333,0.0,99240.000000,30.000000,6.866667,...,9.066667,11.200000,43.000000,0.000000,99253.333333,253.333333,9.066667,11.800000,74.666667,0.666667


In [48]:
mesures_debit.shape

(48216, 27)

In [49]:
mesures = mesures_debit.merge(mesures_meteo,
                             on = "Date",
                             how = "outer")

In [50]:
mesures

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Vitesse_vent_07747,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Direction_vent_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,11.800000,10.000000,61.000000,0.000000,99180.000000,220.000000,9.800000,14.300000,90.000000,0.000000
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,9.066667,10.533333,50.333333,0.000000,99193.333333,240.000000,6.666667,12.900000,78.000000,0.533333
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,9.066667,11.200000,43.000000,0.000000,99253.333333,253.333333,9.066667,11.800000,74.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48211,2020-12-31 14:00:00,42.3,319.0,865.0,2010.0,0.75,2.94,11.70,34.0,27.50,...,2.066667,12.600000,40.666667,0.000000,101170.000000,43.333333,1.700000,10.533333,76.333333,-0.033333
48212,2020-12-31 16:00:00,45.7,311.0,1080.0,2000.0,0.75,2.94,11.60,33.7,43.10,...,2.600000,12.400000,48.000000,0.000000,101130.000000,30.000000,1.300000,10.200000,79.000000,-0.100000
48213,2020-12-31 18:00:00,48.6,301.0,1020.0,1950.0,0.75,2.96,11.60,31.7,34.80,...,2.400000,9.800000,55.333333,0.000000,101143.333333,36.666667,2.366667,8.466667,84.333333,-0.033333
48214,2020-12-31 20:00:00,48.9,292.0,967.0,1910.0,0.75,2.98,11.50,33.1,37.20,...,2.433333,7.566667,63.666667,0.000000,101130.000000,43.333333,2.933333,7.666667,86.666667,0.200000


In [51]:
mesures.to_csv("../Data/Base/Mesures.csv",
              index=False)